In [ ]:
print('ACIT 4610, Portfolio-2024')
print('\nGroup-13')
print('\nparticipate: Thomas, M khan, Harith')

Problem 3: Solving the Vehicle Routing Problem with Time Windows (VRPTW) Using 
Ant Colony Optimization (ACO) and Particle Swarm Optimization (PSO)

Overview 
This exercise focuses on optimizing the delivery routes for a fleet of vehicles using two nature
inspired optimization algorithms: Ant Colony Optimization (ACO) and Particle Swarm 
Optimization (PSO). The objective is to find the most efficient routes for a set of vehicles, 
ensuring that all customers receive their deliveries within specified time windows. You will 
implement both ACO and PSO to solve the Vehicle Routing Problem with Time Windows 
(VRPTW) and compare the effectiveness of these algorithms. 

Problem Description 
The VRPTW is a variant of the Vehicle Routing Problem (VRP), which is crucial in logistics, 
transportation, and supply chain management. In VRPTW, a fleet of vehicles must deliver 
goods to multiple customers. Each customer has a specific time window during which the 
delivery must occur. The challenge is to design routes that minimize the total travel distance 
while ensuring all deliveries meet their respective time constraints.

The primary objective of the VRPTW is to minimize the total distance travelled by all vehicles 
while ensuring that: 
1. Each customer is visited exactly once by one vehicle. 
2. Deliveries occur within the specified time windows. 
3. The total demand on any route does not exceed the vehicle's capacity. 

Key Components 
• Depot: The starting and ending point for all vehicles. 
• Customers: Each customer has a location, a demand for goods, a time window for 
receiving the delivery, and a service time. 
• Vehicles: Each vehicle has a limited capacity and must return to the depot after 
completing its route. 

In [ ]:
# 1. Dataset, and Preprocessing
#https://www.sintef.no/projectweb/top/vrptw/100-customers/